Написание правил для дат (типа «Июль 1993»и т.п.)
В парсере есть только количественные конструкции (пять котов),
необходимо добавить правила для
групп типа пятое июля 1993 года, пятого июля 1993 года и т.д. в падежных формах.


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=b8c1244b078a0767c620e44ba93ef9900a073313b75587a83e6357f38429f663
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import nltk
from nltk import *
import nltk
nltk.download('book_grammars')
import pymorphy2 as pm
import codecs

[nltk_data] Downloading package book_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/book_grammars.zip.


In [ ]:
## загружаем PyMorphy2
m = pm.MorphAnalyzer()
## открываем (создаем)файл с грамматикой, куда будут записываться правила
f = codecs.open("test.fcfg", mode= "w", encoding = "utf-8")
rules = codecs.open("/content/rules_dasha.txt", mode= "r", encoding = "utf-8")
## записываем правила, которые вручную делаем (некоторые на основе правил из АОТ)
for rule in rules:
	f.writelines(rules)

rules.close()
f.close()

In [ ]:
## функция, которая переводит нужную нам информацию из пайморфи в вид, читаемый парсером NLTK
## принимает (токенизированное) словосочетание на входе, записывает правила (lexical productions) в тот же файл с грамматикой

In [ ]:
def pm2fcfg (phrase): ## phrase - это словосочетание, которое мы разбираем
    f = codecs.open("test.fcfg", mode= "a", encoding = "utf-8")
    for x in phrase:
        a = m.parse(x) ## a - список возможных вариантов морфологического разбора слова, предлагаемых пайморфи
		## от части речи зависит, какие признаки отправляются в грамматику, осюда условия
        for y in a:
            if (y.tag.POS == "NOUN") or (y.tag.POS == "ADJF") or (y.tag.POS == "PRTF"):
                strk = str(y.tag.POS) + "[C=" + str(y.tag.case) + ", G=" + str(y.tag.gender) + ", NUM=" + str(y.tag.number) + ", PER=3" + ", NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                f.writelines(strk)
            elif (y.tag.POS == "ADJS") or (y.tag.POS == "PRTS"):
                strk = str(y.tag.POS) + "[G=" + str(y.tag.gender) + ", NUM=" + str(y.tag.number) + ", NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                f.writelines(strk)
            elif (y.tag.POS == "NUMR"):
                strk = str(y.tag.POS) + "[C=" + str(y.tag.case) + ", NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                f.writelines(strk)
            elif (y.tag.POS == "ADVB") or (y.tag.POS == "GRND") or (y.tag.POS == "COMP") or (y.tag.POS == "PRED") or (y.tag.POS == "PRCL") or (y.tag.POS == "INTJ"):
                strk = str(y.tag.POS) + "[NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                f.writelines(strk)
            elif (y.tag.POS == "PREP") or (y.tag.POS == "CONJ"):
                strk = str(y.tag.POS) + "[NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                f.writelines(strk)
                break
            elif (y.tag.POS == "NPRO") & (y.normal_form != "это")& (y.normal_form != "нечего"):
                if ((y.tag.person[0] == "3") & (y.tag.number == "sing")):
                    strk = str(y.tag.POS) + "[C=" + str(y.tag.case) + ", G=" + str(y.tag.gender) + ", NUM=" + str(y.tag.number) + ", PER=" + str(y.tag.person)[0] + ", NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                else:
                    strk = str(y.tag.POS) + "[C=" + str(y.tag.case) + ", NUM=" + str(y.tag.number) + ", PER=" + str(y.tag.person)[0] + ", NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                f.writelines(strk)
            elif (y.tag.POS == "VERB")  or (y.tag.POS == "INFN"):
                if (y.tag.tense == "past"):
                    strk = str(y.tag.POS) + "[TR=" + str(y.tag.transitivity) + ", TENSE=" + str(y.tag.tense) + ", G=" + str(y.tag.gender) + ", NUM=" + str(y.tag.number) + ", PER=" + "0" + ", NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                elif (y.tag.POS == "INFN"):
                    strk = str(y.tag.POS) + "[TR=" + str(y.tag.transitivity) + ", TENSE=0, G=0, NUM=0, PER=0, NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                else:
                    strk = str(y.tag.POS) + "[TR=" + str(y.tag.transitivity) + ", TENSE=" + str(y.tag.tense) + ", G=" + "0" + ", NUM=" + str(y.tag.number) + ", PER=" + str(y.tag.person)[0] + ", NF=u'" + str(y.normal_form) + "'] -> '" + str(y.word) + "'\n"
                f.writelines(strk)
    f.close()

In [ ]:
text = input("введите предложение для разбора: ") ## сюда пишется словосочетание для разбора
words = word_tokenize(text.lower()) ## разбиваем словосочетание на токены

pm2fcfg(words) ## запускаем функцию, описанную выше
cp = load_parser('test.fcfg') ## открываем нашу грамматику, смотрим на разбор в консоли или ещё где
for tree in cp.parse(words):
		print(tree)

введите предложение для разбора: двадцать третье марта


ValueError: ignored